<a href="https://colab.research.google.com/github/DamienSmith/UTS_ML2019_ID13039957/blob/master/A2_PracticalProject_13039957_13026998.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A2: Algorithm Implementation | 31005 | Advanced Data Analytics
Student: 13039957 & 13026998

Link to Github:
https://github.com/DamienSmith/UTS_ML2019_ID13039957/blob/master/A2_PracticalProject_13039957_13026998.ipynb

##Introduction - a project overview (100)


**The Decision Tree (CART) Algorithm** 

A decision tree is a tree in which each branch node represents a choice between a number of alternatives, and each leaf node represents a classification or decision (Singh & Gupta 2014). Starting with a root node rules are applied that split and generate new nodes recursively according to a learning algorithm resulting in a decision tree where each edge represents a question and its outcome (Peng, Chen & Zhou 2009). Common decision tree algorithms include ID3, CART and C4.5 which each use different splitting criteria for splitting a node at each level to find a node with the least possible solutions (Singh & Gupta 2014). 

For this project, the CART (Classification and Regression Trees) algorithm will be implemented from scratch. The representation of the CART model is a binary tree where each node can have zero, one or two child nodes (Brownlee 2016). All input variables and all possible split points are evaluated and chosen in a greedy manner based on the cost function. The Gini cost function is used which provides an purity score which represents how mixed the training data assigned to each node is. A split is made on the best score at each level of depth. Splitting continues until nodes contain a minimum number of training examples or a maximum tree depth is reached. Once created, a tree can be navigated with a new row of data following each branch with the splits until a final prediction is made.

**Define Input/Output** 
* What data you use and what you’re getting out
* the format of the I/O data

The data we are using is the iris dataset, however, this algorithm is designed to handle any dataset as long as it is inputted as a pandas dataframe. Pandas is a Python library providing integrated, intuitive routines for performing common data manipulations and analysis on data sets (McKinney 2011). After running the algorithm, the predicted value of each row of the testing dataframe will be printed in the command line. 


##Exploration (300)

**Identify Challenges**

* Highlight the practical significance of the project
* What problems exist and how to manage them
  * e.g. memory management, time efficiency, 'advanced functions' (eg parallelism) 

**Design Data Structures**

* Design/planning of research/development is clear and logical
* Cover these:
  * data acquisition
  * quality control
  * modelling technicques
  * evaluation method & criteria

**Plan Data Models and Tests**

* Logical design (correct, efficient and practically complete)
* Evaluation method/Testing (Compare and consider alternatives)
* Must be able to run code through collab 
  * i.e. data load and python libraries need to work in Collab


**Possible alternatives**

* Algorithm Tuning. The application of CART to the Bank Note dataset was not tuned. Experiment with different parameter values and see if you can achieve better performance.

* Cross Entropy. Another cost function for evaluating splits is cross entropy (logloss). You could implement and experiment with this alternative cost function.

* Tree Pruning. An important technique for reducing overfitting of the training dataset is to prune the trees. Investigate and implement tree pruning methods.

* Categorical Dataset. The example was designed for input data with numerical or ordinal input attributes, experiment with categorical input data and splits that may use equality instead of ranking.

* Regression. Adapt the tree for regression using a different cost function and method for creating terminal nodes.

* More Datasets. Apply the algorithm to more datasets on the UCI Machine Learning Repository.



There several choices there is historically ID3 the Iterative Dichotomiser 3, C5 which I believe stands for Classifier as development of ID3. CART, which is for classification and regression tree. It's very common in Universal Choice. CHAID, which is for CHi-squared Automatic Interaction Detectior. MARS, which is multivariate adaptive regression splines. And the one that I'm going to be using Conditional Inference trees. Now decision trees in general have some pros and cons.



##Methodology (100 ex comments)

[code here]

**Build and Train Data Models**

* Comments connect the code to the algorithm steps
* 





## Decision Tree Model

In [0]:
## 
##    A2: Algorithm Implementation | 31005 | Advanced Data Analytics
##    
##    Authors: Rae Ho (13026998) & Damien Smith (13039957)
##    Goals: - Implement Decision Tree Algorithm
##           - Build & Train a model
##           - Explore/Compare different parameters 
##    Code: Python 3
##    Github: https://github.com/DamienSmith/UTS_ML2019_ID13039957/blob/master/A2_PracticalProject_13039957_13026998.ipynb
##
##

In [0]:
#Import Libraries
#numpy to work with arrays
#panda for dataframe 
#sklearn to import iris dataset

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

In [0]:
#Get data
#using iris data. Get it from sklearn dataset
dataset = datasets.load_iris()

# We want to turn the dataset into a panda dataframe
df = pd.DataFrame(dataset['data'])


# We also want to add the names of each feature to the dataframe
df.columns = [name[:-5] for name in dataset['feature_names']]

#We want to add the target to the dataframe too 
df['target'] = dataset['target']

#this is to create a random sample of 30 entries
testdf = df.sample(n=30, random_state=1).reset_index(drop=True)


# We create target variable (into binary) in order to get a 'count' of all items in the dataframe (in case there are missing values) 
df['dummy'] = 1

In [0]:
#function to return the count of rows in the dataframe that match a condition

def count_target(dframe, right, feature, value):
  #this is to find the impurity to the left of and to the right of a slice on a cartesian plane   
  if right:
    cond = dframe[feature]>value
  else:
    cond = dframe[feature]<=value
  
  #this works by counting the total number of target items, 'mean' helps find probability of correct choice 
  #count = len(dframe[cond])
  count = sum(cond)
  return count

  

In [0]:
#We want to create a method to determine what the impurity is for any 'slice' of a feature. 
#We have decided to use the gini impurity which shows how likely it is that a randomly selected feature is correctly guessed

##this takes the parameters:
    #dframe which allows you to put in the dataframe you want to find the impurity for
    #right which takes True or False to look at right or left impurity of a split point
    #feature which is the feature (column) that you want to use/look at
    #value which is the value that is to be split at
    #target variable to be determined (which for the iris df we are using is 'target')


#find gini impurity of a slice 
def gini_impurity_leaf(dframe, right, feature, value, target_variable):

#this is to find the impurity to the left of and to the right of a slice on a cartesian plane   
  if right:
    cond = dframe[feature]>value
  else:
    cond = dframe[feature]<=value
  
  #this works by counting the total number of target items, 'mean' helps find probability of correct choice 
  count_of_target = dframe[cond].groupby(target_variable)['dummy'].count()
  #print("count of target is: ")
  #print(count_of_target)
  #this uses the impurity formula of impurity = 1 - sum of (probability of being correct [count/length])^2
  gini_impurity = 1 - (np.divide(count_of_target, len(dframe[cond])) ** 2).sum()
  
  return gini_impurity


In [0]:
#this finds the net gini impurity by adding the left and right slice
# we want to find the 'slice' with the lowest net gini impurity 

def net_gini_impurity(dframe, feature, value, target_variable):  

#  net_gini = gini_impurity_leaf(dframe, True, feature, value, target_variable) + gini_impurity_leaf(dframe, False, feature, value, target_variable) #this didn't work as it would have duplicate values or prefer edge cases
# we had to use a weighted gini index

  right_weight = count_target(dframe, True, feature, value) / len(dframe)
  left_weight = 1 - right_weight
 #print("right weight is:{}".format(right_weight))
  net_gini = (right_weight * gini_impurity_leaf(dframe, True, feature, value, target_variable)) + (left_weight * gini_impurity_leaf(dframe, False, feature, value, target_variable))
  
  return net_gini


In [0]:
# This finds best split 

def find_best_split(dataframe):
# Find the best split by going over every value in each feature and choosing the split with the lowest net impurity
    lowest_impurity = 2  # keep track of the worst impurity
    best_split = ['there is no split', 0]  # keep train of the feature / value that produced it

    
    for f in range(len(dataframe.columns) -2):  # loop through each feature --> the -2 is because it shouldnt look at TARGET and DUMMY
        feat = dataframe.columns[f]             # store the name of the feature as 'feat'
        for val in dataframe[feat].unique():  # loop through each unique value for that feature 
          #Calculate the net impurity of each value in feature
          #print(feat,val)
          net_imp = net_gini_impurity(dataframe, feat, val, 'target')
          #print(feat,val,net_imp)
          split = [feat, val]
          # store best gain and best feature 
          if net_imp < lowest_impurity:
            lowest_impurity = net_imp
            best_split = split
            #print("the current best split is:")
            #print(lowest_impurity,best_split)

    return lowest_impurity, best_split

In [0]:
#this splits a dataframe to right and left sides

def splitter(dframe):
  impurity, featsplit = find_best_split(dframe)
  feature = featsplit[0]
  value = featsplit[1]
  
  #and store the values of those sides into a new data_frame  
  right_split = dframe[dframe[feature]>value]
  left_split =dframe[dframe[feature]<=value]      
  
  print("I have been split for the feature ", feature, "at value", value)
  
  return right_split, left_split, feature, value



In [0]:
#This returns what the target is
# This returns a predicted value for the terminal node
# The returned value is the end value with the most common end result

#def final_guess(feature, value, dframe, boolean=False):
def final_guess(dframe):

  dataset = dframe
  targets = dataset.target
  outcome = None
  
  outcome = targets.value_counts().idxmax()  #this is the first most common value in that set - so if it's a 50/50 split between two targets, it'll choose the first one.
  
  return outcome


  #alternatively you could do 'outcomes = targets.mode()' but then you wont get a single int if there's multiple values

In [0]:
#this creates an depth of arrays which we use to store values for later predict 
def createDepthArray(maxdepth):
  for i in range(1,maxdepth):
    depth = i
    name = 'depth{}'.format(depth)          #get the name of the dict array

    #populate the array with futher arrays
    if name not in globals():       
      globals()['depth%s' % depth] = {}       #create an array called depth#    
    
    if len(globals()['depth%s' % depth]) < 1:
      for n in range(1, 2**depth + 1):              
        globals()['depth%s' % depth][n] = {}
      #elif name in globals():
      #  print("already exists")



In [0]:
#this helps insert values into the previous 'depth array' parts
def insertNodeValue(depth, number, feature, value, side, guess):     
  #number = getNumberPos(depth)
  globals()['depth%s' % depth][number] = {'feature': feature, 'value': value, 'side': side, 'decision': guess}



In [0]:
def depthFiller(depth, outcome, number, maxdepth):
#to fill subsequent depths's numbers that already has a final decision   
  new_depth = depth+1
  #number = getNumberPos(depth)
  if new_depth < maxdepth:

    print("i am at position {} and am filling lower depth".format(number))
    number_left = (number*2 - 1)
    number_right = (number * 2)
    globals()['depth%s' % new_depth][number_right] = {outcome}
    globals()['depth%s' % new_depth][number_left] = {outcome}
    #and then repopulate the sub-depths until max-depth
    depthFiller(new_depth, outcome, number_right, maxdepth)
    depthFiller(new_depth, outcome, number_left, maxdepth)
  

In [0]:
#this returns the next empty number for that depth
def getNumberPos(depth):
  for i in range(1, 2**depth+1):
    if len(globals()['depth%s' % depth][i]) == 0:
      return i
      break
    else:
      continue
    
  
  

In [0]:
#this is the main code that creates the tree by iteratively splitting
#data at the best point

def subsequent_split(dataframe, depth, maxdepth):
  global maxDepth
  maxDepth = maxdepth
  createDepthArray(maxdepth)
  
  right_split, left_split, feature, value = splitter(dataframe)
  #create a bunch of dict to max depth
  

  # store:  [number: {feature = '', value ='', side = 'left', decision = 0,1,3,null}]

  if depth < maxdepth:
    print("I am doing left side")
    
    if left_split.empty:
      print("depth is", depth)
      print("left side is empty")

           
    #this currently looks for a unique value, and if only 1 
    #consider doing a mininum impurity    
    elif len(left_split['target'].unique()) < 2:
      print("depth is", depth)
      print("i am now uniquely left")
      guess = final_guess(left_split)
      print("final guess is {}".format(guess))
      number = getNumberPos(depth)
      depthFiller(depth, guess, number, maxdepth)
      insertNodeValue(depth, number, feature, value, 'left', guess)


    else:  
      #print(len(left_split['target'].unique()))
      print("depth is", depth)
      print("i have split the left side at {}{}".format(feature, value))
      number = getNumberPos(depth)
      insertNodeValue(depth, number, feature, value, 'left', None)
      subsequent_split(left_split,depth+1,maxdepth)
     
    print("I am doing right side")
    if right_split.empty:
      print("depth is", depth)
      print("right side is empty")  
    elif len(right_split['target'].unique()) < 2: 
      print("depth is", depth)
      print("i am now uniquely right")
      guess = final_guess(right_split)
      print("final guess is {}".format(guess))
      number = getNumberPos(depth)
      depthFiller(depth, guess, number, maxdepth)
      insertNodeValue(depth, number, feature, value, 'right', guess)
            
    else:
      print("depth is", depth)
      print("I have split the right side at {}{}".format(feature, value))
      number = getNumberPos(depth)
      insertNodeValue(depth, number, feature, value, 'right', None)
      subsequent_split(right_split,depth+1,maxdepth)
      
      
   
   
 

In [0]:
#def to clear the dictionary. Needed if we want to re-train/subsequent_split  
def clearTrainedData():
  for n in range(1, maxDepth):              
    globals()['depth%s' % n].clear()

In [0]:
#this is for subsequent predictions after the first depth
def subsequentPredict(test_df_row, number, depth):
  index = number
  depth = depth + 1
  if depth < maxDepth:
    for x in test_df_row.columns:               #for each df header            
      if globals()['depth%s' % depth][index]['feature'] == x:           #to see if feature is same
        dictVal = globals()['depth%s' % depth][index]['value']                # value of depth1      
        for n in range(len(test_df_row[x])):     #loop each index row in testdf        
          rowVal = test_df_row.loc[n, x ]   #store the value for that row
      
          if globals()['depth%s' % depth][index]['side'] == 'right':      # if depth entry is left or right
            if rowVal > dictVal:
              if globals()['depth%s' % depth][index]['decision'] is not None:
                print("the prediction is {}".format(globals()['depth%s' % depth][index]['decision']))
              else:
                #print("need to continue")
                subsequentPredict(test_df_row, index*2 - 1, depth)
                subsequentPredict(test_df_row, index*2, depth)
                
          if globals()['depth%s' % depth][index]['side'] == 'left':      # if depth entry is left or right   
            if rowVal <= dictVal:
              if globals()['depth%s' % depth][index]['decision'] is not None:
                print("the prediction is {}".format(globals()['depth%s' % depth][index]['decision']))
              else:
                #print("need to continue")
                subsequentPredict(test_df_row, index*2 - 1, depth)
                subsequentPredict(test_df_row, index*2, depth)
  #else:
  #  print("reached max depth already")
    

            
            
           
        
     




In [0]:
def firstPredict(test_df_row):
  for x in test_df_row.columns:               #for each df header
    for i in range(1,len(depth1)+1):          #loop through each dict key
      if depth1[i]['feature'] == x:           #to see if feature is same
        #print("feature to look over is{}".format(x))
        dictVal = depth1[i]['value']                # value of depth1
        #print("dictval is {}".format(dictVal))
        #print(type(dictVal))
        
        for n in range(len(test_df_row[x])):     #loop each index row in testdf        
          rowVal = test_df_row.loc[n, x ]   #store the value for that row
          #print("rowVal is {}".format(rowVal))
          #print(type(rowVal))
          #print(depth1[i]['side'])
          
          if depth1[i]['side'] == 'right':      # if depth1 entry is left or right
            if rowVal > dictVal:
              #print("Row Val is greater than DictVal")
              if depth1[i]['decision'] is not None:
                print("the prediction is {}".format(depth1[i]['decision']))
              elif depth1[i]['decision'] is None:
                #print("need to continue")
                subsequentPredict(test_df_row, i*2 - 1, 1)
                subsequentPredict(test_df_row, i*2, 1)
            #else:
            #  print("rowVal does not match this node")
          
          if depth1[i]['side'] == 'left':      # if depth1 entry is left or right   
            if rowVal <= dictVal:
              #print("Row Val is less than DictVal")
              if depth1[i]['decision'] is not None:
                print("the prediction is {}".format(depth1[i]['decision']))
              elif depth1[i]['decision'] is None:
                #print("need to continue")
                subsequentPredict(test_df_row, i*2 - 1, 1)
                subsequentPredict(test_df_row, i*2, 1)


In [0]:
#this is to test each row of the inputed dataframe
def testData(dataframe):
  testdata = dataframe
  
  for i in range(len(testdf)):
    testrow = testdata[:1]
    print("for the row:")
    print(testrow.to_string(index=False))
    firstPredict(testrow)
    testdata = testdata.drop([0,0]).reset_index(drop=True)
    print("\n")
  

In [0]:
####################################################
#######this is where we finally call things ########
####################################################

In [0]:
#this is to build the tree
#the first parameter is the training dataframe
#the second parameter is the depth to start at, which is always 1
#the third parameter is the max depth you want to trim the decision tree to
subsequent_split(df, 1, 5)    #this is max depth 5

#You will need to clear the training data if you want to 'train' different data
#clearTrainedData()

In [0]:
#this tests the data you are using
testData(testdf)




##Evaluation (200)


**Report Execution on Data**

**Report Testing**

**Efficiency Analysis**

**Comparative Study**

ideas:
* how does each feature distribution look like? Are there any differences between feature distribution in train and test data?
* are there any meaningful interactions between the features?
* are there outliers and can they be explained?
* are there missing values or duplicates? What are reasons for them?


## Conclusion (100)


Discuss Reflections

Propose Possible Improvements



## Ethical (200)

* Discuss the social/ethical aspect of the project
  * adopt an ethical model (e.g. Ultitarian or Kantian)
* Consider how the technique could be misused

## Video Pitch

[url to video]

**Highlight Challenges and Effort**

* Describe challenges and how the team addressed them.
  * Python Skills
    * Pandas, numpy, others?
    * Dictionary (Hashmap)
  * Code structure
    * Generating 'depth' and 'terminal' flags in dictionary
    * Handling looping (to avoid hardcoding for iris and adding flexibility to use any dataframe)
  * Managing Assignment scope 
    * We initially wanted to run each cost function (ID3, CART and C4.5) (entropy? info gain?)
    * We also wanted to turn it into a Random forest
    * scoping this down into the time we had available was a challenge (while learning)

## References

Brownlee, J., 2016. Master Machine Learning Algorithms: discover how they work and implement them from scratch. Machine Learning Mastery.

McKinney, W., 2011. pandas: a foundational Python library for data analysis and statistics. Python for High Performance and Scientific Computing, 14.

Peng, W., Chen, J. and Zhou, H., 2009. An implementation of ID3-decision tree learning algorithm. From web.arch.usyd.edu.au/wpeng/DecisionTree2.pdf viewed 21/09/2019

Singh, S. & Gupta, P., 2014. Comparative study ID3, cart and C4. 5 decision tree algorithm: a survey. International Journal of Advanced Information Science and Technology (IJAIST), 27(27), pp.97-103.
